In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
HLA_associations_path = os.path.abspath("../")
sys.path.append(HLA_associations_path)

import numpy as np
import pandas as pd
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

import constants, index_tools, data_tools, medical_code_tools
from utilities import *
from analysis import *

import sys
sys.path.append(constants.GRANTHAM_DISTANCE_PATH)

import grantham_distance as gd

In [2]:
sns.set_palette("tab10")
plt.rcParams.update({'figure.max_open_warning': 0})

In [3]:
loaded = False

# Load BioBank Components

In [4]:
DEV_MODE = False
SIGNIFIER = ""

if not loaded:
    (biobank_data_no_HLA_alleles, biobank_index,
     med_code_mapping) = data_tools.load_all_biobank_components(DEV_MODE, signifier=SIGNIFIER)
    loaded = True
    HLA_alleles = data_tools.load_HLA_data()
    biobank_data = biobank_data_no_HLA_alleles.merge(HLA_alleles, how="left", on="eid")

Importing BioBank Index and Data:
Missing 0 biobank index names
Reduced feature set has 284 features.
UK BioBank Data Loaded.
Size: 502536 rows x 284 columns
Elapsed time: 43.8021 seconds


Mapping Medical Codes:   0%|          | 0/269 [00:00<?, ? feature/s]

In [ ]:
illnesses = [
    "multiple sclerosis",
    'infectious mononucleosis / glandular fever / epstein barr virus (ebv)'
]

cancers = ['hodgkins lymphoma / hodgkins disease', "cervical cancer"]

In [8]:
illness = "multiple sclerosis"
# illness = 'hodgkins lymphoma / hodgkins disease'
# illness = 'nasal cavity cancer'
# illness = "osteoarthritis"
# illness = "cervical cancer"
illness = 'infectious mononucleosis / glandular fever / epstein barr virus (ebv)'

# feature = "cancer_code"
feature = "illness_code"

HLA_allele_columns = ["A1", "A2", "B1", "B2", "C1", "C2"]

illness_values = get_illness_value(biobank_data, illness, feature)

illness_alleles = biobank_data.loc[illness_values][HLA_allele_columns].values.ravel()
illness_alleles = np.unique(illness_alleles[~pd.isnull(illness_alleles)])

illness_alleles

array(['A*01:01', 'A*02:01', 'A*02:03', 'A*02:05', 'A*02:06', 'A*02:11',
       'A*03:01', 'A*03:02', 'A*11:01', 'A*23:01', 'A*24:02', 'A*24:03',
       'A*24:07', 'A*25:01', 'A*26:01', 'A*26:08', 'A*29:01', 'A*29:02',
       'A*30:01', 'A*30:02', 'A*31:01', 'A*32:01', 'A*33:01', 'A*33:03',
       'A*34:02', 'A*66:01', 'A*68:01', 'A*68:02', 'A*74:01', 'B*07:02',
       'B*07:05', 'B*08:01', 'B*13:02', 'B*14:01', 'B*14:02', 'B*15:01',
       'B*15:09', 'B*15:17', 'B*15:18', 'B*18:01', 'B*27:02', 'B*27:05',
       'B*35:01', 'B*35:02', 'B*35:03', 'B*35:05', 'B*35:08', 'B*37:01',
       'B*38:01', 'B*39:01', 'B*39:06', 'B*40:01', 'B*40:02', 'B*40:06',
       'B*41:02', 'B*44:02', 'B*44:03', 'B*44:04', 'B*44:05', 'B*45:01',
       'B*47:01', 'B*49:01', 'B*50:01', 'B*51:01', 'B*52:01', 'B*53:01',
       'B*55:01', 'B*56:01', 'B*57:01', 'B*57:03', 'B*58:01', 'C*01:02',
       'C*02:02', 'C*03:02', 'C*03:03', 'C*03:04', 'C*04:01', 'C*05:01',
       'C*06:02', 'C*07:01', 'C*07:02', 'C*07:04', 

In [9]:
text = ""
data = []
for allele in tqdm(illness_alleles):
    allele_loc_cols = [allele[0] + f"{i}" for i in [1, 2]]
    all_allele_zygosity = np.sum(biobank_data[allele_loc_cols] == allele, axis=1)

    odds_ratio, p_value, N, lower_bound, upper_bound = calculate_OR(illness_values, all_allele_zygosity > 1)
    
    data.append([allele, odds_ratio, p_value, N, lower_bound, upper_bound])

    p_value_str = f"{p_value:.2}" if "e" in str(p_value) else f"{p_value:.3f}"
    line = f"{allele}: OR: {odds_ratio:.3f} p-value: {p_value_str}"
    line += f" 95% CI: {lower_bound:.2f} - {upper_bound:.2f}"
    line += f" N: {N:,}"
    line += "\n" if p_value > .05 else " ***\n"
    text += line
data = pd.DataFrame(data, columns=["allele", "odds_ratio", "p_value", "N", "lower_bound", "upper_bound"])
data["hla_loci"] = data["allele"].apply(lambda s: s[0])

  0%|          | 0/91 [00:00<?, ?it/s]

In [10]:
data.query("p_value < .05 and N > 4").sort_values(["p_value"])

,allele,odds_ratio,p_value,N,lower_bound,upper_bound,hla_loci
35,B*15:01,2.611960,0.030468,6,1.328599,5.134984,B
74,C*03:03,2.848813,0.033858,5,1.359258,5.970710,C
0,A*01:01,1.479608,0.042627,30,1.086822,2.014351,A
